## Introduction
The system will provide a simple example of using python to extract information from a website. Note this is part 1 of 4. Participants are not expected to have any experience in python or background in web scraping. However, some understanding of HTML will be useful.

This draws inspiration from http://web.stanford.edu/~zlotnick/TextAsData/Web_Scraping_with_Beautiful_Soup.html

In [ ]:
# Import dependencies (i.e. packages that extend the standard language to perform specific [advance] functionality)
import urllib
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

From analysing the newsday archieve website we see that the URL follows a parsable convention
http://www.newsday.co.tt/archives/YYYY-M-DD.html 
So our general approach will be as follows:
    1. Generate date in the expected form between an ending and starting date
    2. Test to ensure the dates generated are valid. (refine step1 based on results)
    3. Read the content and process based on our goal for scaping the page

In [ ]:
# Step 1 - create a function to generates a list(array) of dates
def genDates(start_date = date.today(), num_days = 3):
    # date_list = [start - timedelta(days=x) for x in range(0, num_days)] # generate a list of dates
    # While we expand the above line for beginners understanding 
    date_list = []
    for d in range(0, num_days):
        temp = start_date - timedelta(days=d)
        date_list.append(temp.strftime('%Y-%-m-%d'))# http://strftime.org/ used a reference
    return date_list

In [ ]:
# Step 2 -Test the generated URL to ensure they point to 
def traverseDates(func, start_date = date.today(), num_days = 3):
    base_url="http://www.newsday.co.tt/archives/"
    dates_str_list = genDates(start_date, num_days)
    for date in dates_str_list:
        url = base_url + date
        func(url)
        
def printDate(date):
    print(date)
    


In [ ]:
# Step 3 - Read content and process page
def processPage(page_url):
    print("Reading content from {0}".format(page_url))
    page_content = urllib.urlopen(page_url).read()
    beau = BeautifulSoup(page_content, "html5lib")
    
    

In [ ]:
traverseDates(processPage,num_days = 1)